### Importing all the libraries

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

### Using CUDA if it is available

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### Model Parameters

In [3]:
input_size = 32
sequence_length = 96
num_layers = 2
hidden_size = 256
num_classes = 10
learning_rate = 0.0001
batch_size = 64
num_epochs = 2

### Creating a Recurrent Neural Network Model

In [4]:
class LSTM(nn.Module):
    
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(LSTM,self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size*sequence_length, num_classes)
        
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        # Forward Propagation
        out, _ = self.lstm(x, (h0,c0))
        out = out.reshape(x.shape[0],-1)
        out = self.fc(out)
        return out

### Checking the model on random data

In [5]:
model = LSTM(32,256,2,10)
x = torch.randn(64,3,32,32)
x = x.view(64,1,96,32)
x = x.squeeze(1)
print(model(x).shape)

torch.Size([64, 10])


In [6]:
arr = model(x)

In [7]:
arr

tensor([[-1.4499e-02, -6.0854e-04, -9.4485e-03,  7.4918e-03, -2.3024e-02,
          5.2489e-03, -1.3159e-02,  1.0678e-02, -4.5868e-03, -3.3060e-03],
        [-1.7380e-02,  2.1766e-03, -8.4910e-03,  1.6562e-02, -1.5204e-02,
          9.4150e-03, -7.4481e-03,  1.0093e-02, -9.0586e-03, -1.1713e-02],
        [-1.4059e-02, -5.0177e-03, -2.0294e-02,  1.2504e-02, -9.4399e-03,
          1.2521e-02,  7.1549e-04,  1.7123e-02,  1.1859e-02, -2.5747e-02],
        [-1.6484e-02,  9.8910e-03, -2.2535e-02,  2.6131e-02, -1.9626e-02,
          7.5717e-03, -8.7001e-03,  3.2235e-03,  1.5401e-02, -1.3365e-02],
        [-2.2660e-02, -8.9296e-03, -2.1598e-02,  1.7321e-02, -1.6751e-02,
          8.3844e-03,  7.2011e-03,  1.9935e-02, -9.5141e-03, -1.5437e-02],
        [-2.4716e-02,  4.8425e-03, -5.1449e-03,  1.1560e-02,  4.7828e-03,
          9.8432e-03, -1.4218e-02,  1.2663e-02, -9.2092e-03,  3.3147e-03],
        [-1.0044e-02,  9.7843e-03, -1.9395e-02,  1.0526e-03,  3.2787e-03,
          9.7046e-03, -8.4073e-0

In [8]:
torch.einsum("ij->i",arr)

tensor([-0.0452, -0.0310, -0.0198, -0.0185, -0.0420, -0.0063,  0.0020, -0.0137,
        -0.0547,  0.0131, -0.0230, -0.0126, -0.0382, -0.0305, -0.0100, -0.0214,
         0.0097,  0.0257, -0.0325, -0.0441, -0.0482, -0.0200, -0.0126, -0.0101,
         0.0209, -0.0229, -0.0014, -0.0596,  0.0035, -0.0195, -0.0549, -0.0017,
        -0.0164, -0.0528, -0.0086, -0.0193, -0.0681, -0.0622,  0.0183,  0.0193,
        -0.0175, -0.0273, -0.0508, -0.0057, -0.0015, -0.0043,  0.0211, -0.0144,
        -0.0101, -0.0139, -0.0341,  0.0281,  0.0344, -0.0284, -0.0225, -0.0251,
        -0.0123, -0.0111, -0.0014, -0.0212,  0.0153, -0.0252,  0.0021, -0.0107],
       grad_fn=<SumBackward1>)

### Data Loading

In [9]:
train_dataset = datasets.CIFAR10(root='cifar_data/',train=True, transform=transforms.ToTensor(),download=True)
train_loader = DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle=True)
test_dataset = datasets.CIFAR10(root='cifar_data/',train=False, transform=transforms.ToTensor(),download=True)
test_loader = DataLoader(dataset=test_dataset,batch_size=batch_size,shuffle=True)

Files already downloaded and verified
Files already downloaded and verified


### Intialize the Model

In [10]:
model = LSTM(input_size,hidden_size,num_layers,num_classes).to(device)

### Loss and Optimizer

In [11]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

### Train Network

In [12]:
for epoch in range(num_epochs):
    
    for batch_idx, (data, targets) in enumerate(train_loader):
        # Get the data to CUDA if possible
        data = data.to(device=device).view(data.shape[0],1,96,32).squeeze(1)
        targets = targets.to(device=device)
        
        # forward
        scores = model(data)
        loss = criterion(scores,targets)
        
        #backward
        optimizer.zero_grad()
        loss.backward()
        
        # Gradient Descent
        optimizer.step()

### Checking the accuracy of the Model

In [13]:
def check_accuracy(loader, model):
    
    if loader.dataset.train:
        print("Checking accuracy on training data")
        
    else :
        print("Checking accuracy on test data")
    
    num_correct = 0
    num_samples = 0
    
    n_class_correct = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]
    #model.eval()
    
    classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
        
    with torch.no_grad():
        for x,y in loader:
            x = x.to(device=device).view(x.shape[0],1,96,32).squeeze(1)
            y = y.to(device=device)
            
            scores = model(x)
            
            _, predictions = scores.max(1)
            num_correct += (predictions==y).sum()
            num_samples += predictions.size(0)
            
            for i in range(y.size(0)):
                label = y[i]
                pred = predictions[i]
                if (label == pred):
                    n_class_correct[label] += 1
                n_class_samples[label] += 1

    for i in range(10):
        acc = 100.0 * n_class_correct[i] / n_class_samples[i]
        print(f'Accuracy of {classes[i]}: {acc} %')
            
    print(f'Got {num_correct}/{num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}')
        
    #model.train()

check_accuracy(train_loader,model)
check_accuracy(test_loader,model)

Checking accuracy on training data
Accuracy of plane: 45.98 %
Accuracy of car: 46.88 %
Accuracy of bird: 41.5 %
Accuracy of cat: 32.26 %
Accuracy of deer: 30.08 %
Accuracy of dog: 33.36 %
Accuracy of frog: 53.02 %
Accuracy of horse: 58.14 %
Accuracy of ship: 62.16 %
Accuracy of truck: 61.36 %
Got 23237/50000 with accuracy 46.47
Checking accuracy on test data
Accuracy of plane: 45.7 %
Accuracy of car: 44.9 %
Accuracy of bird: 42.6 %
Accuracy of cat: 31.2 %
Accuracy of deer: 27.4 %
Accuracy of dog: 33.7 %
Accuracy of frog: 57.2 %
Accuracy of horse: 57.4 %
Accuracy of ship: 62.6 %
Accuracy of truck: 62.3 %
Got 4650/10000 with accuracy 46.50
